Core tasks : run the API, get results, store data in a dataframe, vary temperature to get different responses.
Token counts should be monitored to avoid exceeding the limit.
    
Remaining tasks:
error handling
testing for speed

In [25]:
import openai
from project_secrets import *   # API key stored in project_secrets.py
import pandas as pd

openai.api_key = OPENAI_API_KEY

def get_response(prompt, data):
    for i in range(0, 11, 2):
       temperature = i / 10
       response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=temperature, max_tokens=15)
       new_temp = {"temp": temperature}
       response.update(new_temp)
       data.append(response)

def store_response(data):
    normalized = pd.json_normalize(data)
    df = pd.DataFrame()
    df['id'] = normalized['id']
    df['temperature'] = normalized['temp']
    df['text'] = normalized['choices'][0][0]['text'].strip()
    df['logprobs'] = normalized['choices'][0][0]['logprobs']
    df['completion_tokens'] = normalized['usage.completion_tokens']
    df['prompt_tokens'] = normalized['usage.prompt_tokens']
    df['total_tokens'] = normalized['usage.total_tokens']
    return df

In [26]:
# First run - vary temperature on each loop
data = []
prompt = "Say something like, 'this is only a test', but change the output"
get_response(prompt, data)

foo = store_response(data)
print(foo)

                                   id  temperature                       text  \
0  cmpl-79Wy2rKgW3ss8BjdYKOGfI5vfsLcs          0.0  This is just a trial run.   
1  cmpl-79Wy3cVPzdtnnJl6R6mRl0JAuujpX          0.2  This is just a trial run.   
2  cmpl-79Wy47gXPBi8RPlnBqDkV9lE8fkop          0.4  This is just a trial run.   
3  cmpl-79Wy6jIh2zW2x4YZEUBI6XNmjCKUe          0.6  This is just a trial run.   
4  cmpl-79Wy7OIvhc30PVDrTL1saZZQNZXMx          0.8  This is just a trial run.   
5  cmpl-79Wy9fbVLKBhGacodhNnbPgxzp8pz          1.0  This is just a trial run.   

  logprobs  completion_tokens  prompt_tokens  total_tokens  
0     None                  9             15            24  
1     None                  9             15            24  
2     None                  9             15            24  
3     None                  9             15            24  
4     None                  8             15            23  
5     None                  8             15            23  


In [27]:
# Expected: different results at different temperature. 
# Actual: same result at all temperatures.
# Conclusion: temperature seems to not change across requests.
# Next step: Keep changing temperature, but change prompt to ask for more responses with each API call. This requires more max_tokens to capture full set

def get_response(prompt):
    for i in range(0, 11, 2):
        temperature = i / 10
        response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=temperature, max_tokens=30)
        print(f"Temperature: {temperature}.  Response: {response.choices[0].text.strip()}")

prompt="Say something like, 'this is only a test', but give me two different versions."
get_response(prompt)

Temperature: 0.0.  Response: 1. "This is merely a trial run."
2. "This is just a practice run."
Temperature: 0.2.  Response: 1. "This is merely a test."
2. "This is just a trial run."
Temperature: 0.4.  Response: 1. "This is merely a trial run." 
2. "This is just a practice run."
Temperature: 0.6.  Response: 1. "This is just a trial run."
2. "This is merely a practice run."
Temperature: 0.8.  Response: 1. "I'm just running a test here."
2. "I'm only trying this out."
Temperature: 1.0.  Response: 1. I'm just testing this out.
2. Let's see how this works.
